In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten
import pickle

In [4]:
IoT = pd.read_csv('../../data/IoT_Weather.csv')
IoT = IoT.iloc[0:500]

IoT = IoT.dropna()

encoder=LabelEncoder()
IoT['type']=encoder.fit_transform(IoT['type'])


IoT.head()

,date,time,temperature,pressure,humidity,label,type
0,31-Mar-19,12:36:52,31.788508,1.035,32.036579,0,0
1,31-Mar-19,12:36:53,41.630997,1.035,30.886165,0,0
2,31-Mar-19,12:36:54,42.256959,1.035,19.755908,0,0
3,31-Mar-19,12:36:55,49.116581,1.035,78.949621,0,0
4,31-Mar-19,12:36:56,24.017085,1.035,40.001059,0,0


In [5]:
x = IoT.drop(['label', 'date', 'time','type'], axis=1)
y = IoT['label']

features=len(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [6]:
model1 = Sequential()
    
model1.add(LSTM(units = 128, activation='tanh', return_sequences=True, input_shape = (features,1)))
model1.add(Dropout(0.2))
model1.add(LSTM(units = 64, activation='tanh', return_sequences=True))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid')) 
model1.add(Flatten())
    
model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
model1.save('../h5s/LSTM.h5') 
# filename = '../h5s/LSTM.h5'
# pickle.dump(model1, open(filename, 'wb'))

In [8]:
model1.fit(x_train, np.asarray(y_train).astype('float32').reshape((-1,1)), epochs = 5, batch_size=32, validation_data=(x_test, np.asarray(y_test).astype('float32').reshape((-1,1))))

Epoch 1/5
13/13 [==============================] - 5s 81ms/step - loss: 0.2665 - accuracy: 0.9175 - val_loss: 0.0748 - val_accuracy: 1.0000
Epoch 2/5
13/13 [==============================] - 0s 11ms/step - loss: 0.0488 - accuracy: 1.0000 - val_loss: 0.0230 - val_accuracy: 1.0000
Epoch 3/5
13/13 [==============================] - 0s 10ms/step - loss: 0.0166 - accuracy: 1.0000 - val_loss: 0.0081 - val_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 0s 11ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 5/5
13/13 [==============================] - 0s 10ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000


In [9]:
predictions = model1.predict(x_test)
y_pred = [round(x[0]) for x in predictions]
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [10]:
for i in range(3):
	print(x_test.values[i].tolist(), '=>', y_pred[i], '( expected ', y_train.values[i], ')')

[48.94736504, 0.599772085, 78.94962111] => 0 ( expected  0 )
[28.76875922, 1.035, 11.56575114] => 0 ( expected  0 )
[39.04501695, 1.035, 76.76133899] => 0 ( expected  0 )


In [11]:
score, acc = model1.evaluate(x_test, y_test, batch_size=1)

print('Test score:', score)
print('Test accuracy:', acc)

100/100 [==============================] - 1s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Test score: 0.0014442759566009045
Test accuracy: 1.0
